The tire data processing notebook
Franco Antunez
All the data used comes from Calspan and the Formual SAE TTC

The first section of the code is the import statements for all of the different libraries that are used in the code.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import curve_fit
import numpy as np
from numpy import arange
from sympy import *
from scipy.signal import savgol_filter 
import xlsxwriter
import scipy.io as sio
import csv
from tkinter import *
from tkinter import ttk
from tkinter import filedialog

After the import section of the code, there is the code related to creating the simple GUI with the tkinter library. The code chunk builds the GUI and formats the buttons and input boxes that will save the tire file choices that you make. The functions are used to save the values that are chosen by the user so that they can be used later in the code, take note of the global variables.

In [ ]:
interface = Tk()

def LateralFile():
    global lateral
    lateral = filedialog.askopenfilename()
    return lateral

def LongFile():
    global longitude
    longitude = filedialog.askopenfilename()
    return longitude

def Outputfolder():
    global output
    output = filedialog.askdirectory()
    return output

def camberchoice():
    global camber
    global outName
    global psi
    camber = int(a.get())
    psi = int(p.get())
    outName = b.get()
    interface.destroy()


button1 = ttk.Button(interface, text="Lateral File", command=LateralFile)  # <------
button1.grid(column=100, row=10)


button1 = ttk.Button(interface, text="Longitudinal File", command=LongFile)  # <------
button1.grid(column=100, row=30)


button1 = ttk.Button(interface, text="Output Folder", command=Outputfolder)  # <------
button1.grid(column=100, row=50)

label = ttk.Label(interface, text = "Camber Choice 0, 2, or 4", font =('Calibri 12'))
label.grid(column = 100, row = 70)
a = Entry(interface, width = 15)
a.grid(column=100,row = 90)

label = ttk.Label(interface, text = "Pressure Choice 10, 12, or 14", font =('Calibri 12'))
label.grid(column = 100, row = 110)
p = Entry(interface, width = 15)
p.grid(column=100,row = 130)

label = ttk.Label(interface, text = "Tire Output File Name", font =('Calibri 12'))
label.grid(column = 100, row = 150)
b = Entry(interface, width = 15)
b.grid(column=100,row = 170)

button1 = ttk.Button(interface, text="Set Choices", command=camberchoice)  # <------
button1.grid(column=100, row=190)

interface.mainloop()

interface = Tk()

label = ttk.Label(interface, text = 'Lateral Force File', font =('Calibri 12'))
label.grid(column = 100, row = 10)
label = ttk.Label(interface, text = lateral, font =('Calibri 10'))
label.grid(column = 100, row = 20)

label = ttk.Label(interface, text = 'Longitudinal Tire File', font =('Calibri 12'))
label.grid(column = 100, row = 30)
label = ttk.Label(interface, text = longitude, font =('Calibri 10'))
label.grid(column = 100, row = 40)

label = ttk.Label(interface, text = 'Output Folder', font =('Calibri 12'))
label.grid(column = 100, row = 50)
label = ttk.Label(interface, text = output, font =('Calibri 10'))
label.grid(column = 100, row = 60)

label = ttk.Label(interface, text = 'Camber Coice', font =('Calibri 12'))
label.grid(column = 100, row = 70)
label = ttk.Label(interface, text = camber, font =('Calibri 10'))
label.grid(column = 100, row = 80)

label = ttk.Label(interface, text = 'Pressure Coice', font =('Calibri 12'))
label.grid(column = 100, row = 90)
label = ttk.Label(interface, text = psi, font =('Calibri 10'))
label.grid(column = 100, row = 100)

label = ttk.Label(interface, text = 'Output File Name', font =('Calibri 12'))
label.grid(column = 100, row = 110)
label = ttk.Label(interface, text = outName, font =('Calibri 10'))
label.grid(column = 100, row = 120)
interface.mainloop()


After the tkinter GUI is made the code will go through the process of properly importing and reformating the .mat CORNERING tire files that you chose earlier. This feels like one of the more repetative setions in the code as the way that matlab saves data files is somewhat complicated. also note that seaborn is used for graph formating

In [ ]:
sns.set_style("ticks")
# These headers are grafted onto the csv you input. they match the matlab script output
def lighten_color(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

# This is the lateral load file input. download the respective .mat cornering file from the ttc 
# the data channels can be changed if you really want to

LatLoads = sio.loadmat(lateral)
def Convert(string):
    li = list(string.split(","))
    out = []
    for item in li:
        out.append(float(item))
    return out

ETin = LatLoads['ET'].tolist()
ETin = str(ETin)
ETin = ETin.replace("[","")
ETin = ETin.replace("]","")
ETin = ETin.replace(" ","")
ETin = Convert(ETin)

FXin = LatLoads['FX'].tolist()
FXin = str(FXin)
FXin = FXin.replace("[","")
FXin = FXin.replace("]","")
FXin = FXin.replace(" ","")
FXin = Convert(FXin)

FYin = LatLoads['FY'].tolist()
FYin = str(FYin)
FYin = FYin.replace("[","")
FYin = FYin.replace("]","")
FYin = FYin.replace(" ","")
FYin = Convert(FYin)

FZin = LatLoads['FZ'].tolist()
FZin = str(FZin)
FZin = FZin.replace("[","")
FZin = FZin.replace("]","")
FZin = FZin.replace(" ","")
FZin = Convert(FZin)

SAin = LatLoads['SA'].tolist()
SAin = str(SAin)
SAin = SAin.replace("[","")
SAin = SAin.replace("]","")
SAin = SAin.replace(" ","")
SAin = Convert(SAin)

Pin = LatLoads['P'].tolist()
Pin = str(Pin)
Pin = Pin.replace("[","")
Pin = Pin.replace("]","")
Pin = Pin.replace(" ","")
Pin = Convert(Pin)

TSTIin = LatLoads['TSTI'].tolist()
TSTIin = str(TSTIin)
TSTIin = TSTIin.replace("[","")
TSTIin = TSTIin.replace("]","")
TSTIin = TSTIin.replace(" ","")
TSTIin = Convert(TSTIin)

TSTCin = LatLoads['TSTC'].tolist()
TSTCin = str(TSTCin)
TSTCin = TSTCin.replace("[","")
TSTCin = TSTCin.replace("]","")
TSTCin = TSTCin.replace(" ","")
TSTCin = Convert(TSTCin)

TSTOin = LatLoads['TSTO'].tolist()
TSTOin = str(TSTOin)
TSTOin = TSTOin.replace("[","")
TSTOin = TSTOin.replace("]","")
TSTOin = TSTOin.replace(" ","")
TSTOin = Convert(TSTOin)

AMBTMPin = LatLoads['AMBTMP'].tolist()
AMBTMPin = str(AMBTMPin)
AMBTMPin = AMBTMPin.replace("[","")
AMBTMPin = AMBTMPin.replace("]","")
AMBTMPin = AMBTMPin.replace(" ","")
AMBTMPin = Convert(AMBTMPin)

IAin = LatLoads['IA'].tolist()
IAin = str(IAin)
IAin = IAin.replace("[","")
IAin = IAin.replace("]","")
IAin = IAin.replace(" ","")
IAin = Convert(IAin)

d = {'ET' : ETin, 'FX' : FXin, 'FY' : FYin, 'FZ' : FZin, 'SA' : SAin, 'P' : Pin, 'TSTI' : TSTIin,
      'TSTC' : TSTCin, 'TSTO' : TSTOin, 'AMBTMP' : AMBTMPin, 'IA' : IAin}

df = pd.DataFrame(data=d)

The next section is short and is only making a writeable excel file that is used throughout the code for data output, you will also input the name of this file when you start the program.

In [ ]:
# df.to_csv(output + '/df.csv',sep = ',',index = False)
# creates the output file for the tire data to be used later
writer = pd.ExcelWriter(output + '/' + outName + '.xlsx', engine = 'xlsxwriter')
workbook = writer.book


# df.to_csv('C:\\FSAE EV\\Vehicle Dynamics\\TIRES DINK DONKERY\\pandasOUT\\df.csv',sep = ',',index = False)


The following section is the first of the tire data collection, it takes the edited dataframe for the cornering data .mat file and pulls all of the data that fit within the input parameters that were chosen in the GUI. You can see how the GUI choices are used in the sturcture of the loops, This section also feels very repetative but I think its necessary or im just not good enought at this yet. There is also a small section of code at the end of the loop that trims some data that might skew with the curve fitting, if your curves look too short or there is large spikes on some of the curves you need to tweak the data trimming numbers and try again and you will get a clean output.

In [ ]:
# This loop iterates through the tire data csv and pulls the entire row at forces and pressures
for rows in df:
    # dfLatLoads = df.loc[(df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= -1.0) & (df['IA'] <= -1.0)]
    if psi == 10:
    # 0 IA data scrape
        if camber == 0:
            df1 = df.loc[(df['FZ'] >= -55.0) & (df['FZ'] <= -45.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)]

            df2 = df.loc[(df['FZ'] >= -105.0) & (df['FZ'] <= -95.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)]

            df3 = df.loc[(df['FZ'] >= -155.0) & (df['FZ'] <= -145.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)] 

            df4 = df.loc[(df['FZ'] >= -205.0) & (df['FZ'] <= -195.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)]

            df5 = df.loc[(df['FZ'] >= -255.0) & (df['FZ'] <= -245.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)]

    #  2 IA data scrape
        if camber == 2:
            df1 = df.loc[(df['FZ'] >= -55.0) & (df['FZ'] <= -45.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)]

            df2 = df.loc[(df['FZ'] >= -105.0) & (df['FZ'] <= -95.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)]

            df3 = df.loc[(df['FZ'] >= -155.0) & (df['FZ'] <= -145.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)] 

            df4 = df.loc[(df['FZ'] >= -205.0) & (df['FZ'] <= -195.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)]

            df5 = df.loc[(df['FZ'] >= -255.0) & (df['FZ'] <= -245.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)]

    #  4 IA data scrape
        if camber == 4:
            df1 = df.loc[(df['FZ'] >= -55.0) & (df['FZ'] <= -45.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)]

            df2 = df.loc[(df['FZ'] >= -105.0) & (df['FZ'] <= -95.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)]

            df3 = df.loc[(df['FZ'] >= -155.0) & (df['FZ'] <= -145.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)] 

            df4 = df.loc[(df['FZ'] >= -205.0) & (df['FZ'] <= -195.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)]

            df5 = df.loc[(df['FZ'] >= -255.0) & (df['FZ'] <= -245.0) & (df['P'] >= 9.5) & (df['P'] <= 10.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)]
    if psi == 12:
    # 0 IA data scrape
        if camber == 0:
            df1 = df.loc[(df['FZ'] >= -55.0) & (df['FZ'] <= -45.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)]

            df2 = df.loc[(df['FZ'] >= -105.0) & (df['FZ'] <= -95.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)]

            df3 = df.loc[(df['FZ'] >= -155.0) & (df['FZ'] <= -145.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)] 

            df4 = df.loc[(df['FZ'] >= -205.0) & (df['FZ'] <= -195.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)]

            df5 = df.loc[(df['FZ'] >= -255.0) & (df['FZ'] <= -245.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)]

    #  2 IA data scrape
        if camber == 2:
            df1 = df.loc[(df['FZ'] >= -55.0) & (df['FZ'] <= -45.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)]

            df2 = df.loc[(df['FZ'] >= -105.0) & (df['FZ'] <= -95.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)]

            df3 = df.loc[(df['FZ'] >= -155.0) & (df['FZ'] <= -145.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)] 

            df4 = df.loc[(df['FZ'] >= -205.0) & (df['FZ'] <= -195.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)]

            df5 = df.loc[(df['FZ'] >= -255.0) & (df['FZ'] <= -245.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)]

    #  4 IA data scrape
        if camber == 4:
            df1 = df.loc[(df['FZ'] >= -55.0) & (df['FZ'] <= -45.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)]

            df2 = df.loc[(df['FZ'] >= -105.0) & (df['FZ'] <= -95.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)]

            df3 = df.loc[(df['FZ'] >= -155.0) & (df['FZ'] <= -145.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)] 

            df4 = df.loc[(df['FZ'] >= -205.0) & (df['FZ'] <= -195.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)]

            df5 = df.loc[(df['FZ'] >= -255.0) & (df['FZ'] <= -245.0) & (df['P'] >= 11.5) & (df['P'] <= 12.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)]
    
    if psi == 14:
    # 0 IA data scrape
        if camber == 0:
            df1 = df.loc[(df['FZ'] >= -55.0) & (df['FZ'] <= -45.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)]

            df2 = df.loc[(df['FZ'] >= -105.0) & (df['FZ'] <= -95.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)]

            df3 = df.loc[(df['FZ'] >= -155.0) & (df['FZ'] <= -145.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)] 

            df4 = df.loc[(df['FZ'] >= -205.0) & (df['FZ'] <= -195.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)]

            df5 = df.loc[(df['FZ'] >= -255.0) & (df['FZ'] <= -245.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= -1.0) & (df['IA'] <= 1.0)]

    #  2 IA data scrape
        if camber == 2:
            df1 = df.loc[(df['FZ'] >= -55.0) & (df['FZ'] <= -45.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)]

            df2 = df.loc[(df['FZ'] >= -105.0) & (df['FZ'] <= -95.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)]

            df3 = df.loc[(df['FZ'] >= -155.0) & (df['FZ'] <= -145.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)] 

            df4 = df.loc[(df['FZ'] >= -205.0) & (df['FZ'] <= -195.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)]

            df5 = df.loc[(df['FZ'] >= -255.0) & (df['FZ'] <= -245.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= 1.0) & (df['IA'] <= 3.0)]

    #  4 IA data scrape
        if camber == 4:
            df1 = df.loc[(df['FZ'] >= -55.0) & (df['FZ'] <= -45.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)]

            df2 = df.loc[(df['FZ'] >= -105.0) & (df['FZ'] <= -95.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)]

            df3 = df.loc[(df['FZ'] >= -155.0) & (df['FZ'] <= -145.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)] 

            df4 = df.loc[(df['FZ'] >= -205.0) & (df['FZ'] <= -195.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)]

            df5 = df.loc[(df['FZ'] >= -255.0) & (df['FZ'] <= -245.0) & (df['P'] >= 13.5) & (df['P'] <= 14.5) & (df['IA'] >= 3.0) & (df['IA'] <= 5.0)]

df1 = df1.iloc[800:]
df2 = df2.iloc[800:]
df3 = df3.iloc[500:]
df4 = df4.iloc[500:]
df5 = df5.iloc[500:]

# df1.to_csv('C:\\FSAE EV\\Vehicle Dynamics\\TIRES DINK DONKERY\\pandasOUT\\df2.csv',sep = '\t',index = False)
# plt.plot(dfLatLoads['ET'].values, dfLatLoads['FZ'].values)
# plt.show()


After data is pulled with the above loop, the first section of Pacejka curve fitting is used, note that there are 5 separate functions each with slightly different initial guess coefficents, the code will take to long or will run into errors with not enough iterations when there was only one function so i added more to help with runtime and consitency.

In [ ]:

# df3 = df3.iloc[1000:]
# Graphing data
# The four different pacejka fitting algorithms are to make the loading times shorter and so that the curves fit a bit better that if there was
# just one model
def pacejka1(a, B, C, D, E):
    return D * np.sin(C * np.arctan(B * a - E * (B * a - np.arctan(B * a))))

def fit_pacejka1(a, Fy):
    initial_guess = [0.33, 0.93, 50, 0.29]  # initial guess for B, C, D, and E
    popt, _ = curve_fit(pacejka1, a, Fy, p0=initial_guess, maxfev= 100000)
    return popt

def pacejka2(a, B, C, D, E):
    return D * np.sin(C * np.arctan(B * a - E * (B * a - np.arctan(B * a))))

def fit_pacejka2(a, Fy):
    initial_guess = [0.25, 1.03, 100, 0.3018]  # initial guess for B, C, D, and E
    popt, _ = curve_fit(pacejka2, a, Fy, p0=initial_guess, maxfev= 100000)
    return popt

def pacejka3(a, B, C, D, E):
    return D * np.sin(C * np.arctan(B * a - E * (B * a - np.arctan(B * a))))

def fit_pacejka3(a, Fy):
    initial_guess = [0.165, 1.47, 150, 0.3727]  # initial guess for B, C, D, and E
    popt, _ = curve_fit(pacejka3, a, Fy, p0=initial_guess, maxfev= 100000)
    return popt

def pacejka4(a, B, C, D, E):
    return D * np.sin(C * np.arctan(B * a - E * (B * a - np.arctan(B * a))))

def fit_pacejka4(a, Fy):
    initial_guess = [0.187, 0.88, 200, -3.26]  # initial guess for B, C, D, and E
    popt, _ = curve_fit(pacejka4, a, Fy, p0=initial_guess, maxfev= 100000)
    return popt

def pacejka5(a, B, C, D, E):
    return D * np.sin(C * np.arctan(B * a - E * (B * a - np.arctan(B * a))))

def fit_pacejka5(a, Fy):
    initial_guess = [0.18, .00464, 250, -7.09]  # initial guess for B, C, D, and E
    popt, _ = curve_fit(pacejka5, a, Fy, p0=initial_guess, maxfev= 1000000)
    return popt

After we create the curve fitting functions we will begin to fit specific data. The values are curve fit and the coeffients that are deriven are saved for each data set, a graph is also made and limits the data outputs to 200 values for the curve. two separate plots are made on with the curve fit ideal data and one at 65% efficency(closer to real world force values). At the end the data is outputted to excel to a new sheet.

In [ ]:

# This is where the data from the loop is fit to the pacejka model

# Pulls out the slip angle and lateral force values to be fit
a = df1["SA"].values
Fy = df1["FY"].values * (-1)


# the coeff are saved into their own variable to used on the cornering stiffness calculations
# fitting of the model and output of the coeffiecents 
popt = fit_pacejka1(a, Fy)
popt1 = popt
# Plots the fit data 
a_fit1 = np.linspace(min(a), max(a), num=200)
Fy_fit1 = pacejka1(a_fit1, *popt)
Fy_fit_50 = Fy_fit1
plt.plot(a_fit1, Fy_fit1, label='50 lbs', color = 'blue')
plt.plot(a_fit1, Fy_fit1 * 0.65, label='50 lbs scaled', color = lighten_color('blue', 0.4))

a = df2["SA"].values
Fy = df2["FY"].values * (-1)

popt = fit_pacejka2(a, Fy)
popt2 = popt

a_fit2 = np.linspace(min(a), max(a), num=200)
Fy_fit2 = pacejka2(a_fit2, *popt)
Fy_fit_100 = Fy_fit2
plt.plot(a_fit2, Fy_fit2, label='100 lbs', color = 'purple')
plt.plot(a_fit2, Fy_fit2 * 0.65, label='100 lbs scaled', color = lighten_color('purple', 0.4))

a = df3["SA"].values
Fy = df3["FY"].values * (-1)

popt = fit_pacejka3(a, Fy)
popt3 = popt

a_fit3 = np.linspace(min(a), max(a), num=200)
Fy_fit3 = pacejka3(a_fit3, *popt)
Fy_fit_150 = Fy_fit3
plt.plot(a_fit3, Fy_fit3, label='150 lbs', color = 'green')
plt.plot(a_fit3, Fy_fit3 * 0.65, label='150 lbs scaled', color = lighten_color('green', 0.4))

a = df4["SA"].values
Fy = df4["FY"].values * (-1)

popt = fit_pacejka4(a, Fy)
popt4 = popt

a_fit4 = np.linspace(min(a), max(a), num=200)
Fy_fit4 = pacejka4(a_fit4, *popt)
Fy_fit_200 = Fy_fit4
plt.plot(a_fit4, Fy_fit4, label='200 lbs', color = 'orange')
plt.plot(a_fit4, Fy_fit4 * 0.65, label='200 lbs scaled', color = lighten_color('orange', 0.4))

a = df5["SA"].values
Fy = df5["FY"].values * (-1)

popt = fit_pacejka5(a, Fy)
popt5 = popt

a_fit5 = np.linspace(min(a), max(a), num=200)
Fy_fit5 = pacejka5(a_fit5, *popt)
Fy_fit_250 = Fy_fit5
plt.plot(a_fit5, Fy_fit5, label='250 lbs', color = 'red')
plt.plot(a_fit5, Fy_fit5 * 0.65, label='250 lbs scaled', color = lighten_color('red', 0.4))


LatData = {"SA 50" : a_fit1, "FY 50" : Fy_fit1* 0.65,"SA 100" : a_fit2, "FY 100" : Fy_fit2* 0.65,"SA 150" : a_fit3, "FY 150" : Fy_fit3* 0.65, 
          "SA 200" : a_fit4, "FY 200" : Fy_fit4 * 0.65 ,"SA 250" : a_fit5, "FY 250" : Fy_fit5* 0.65}
df_FYSA = pd.DataFrame.from_dict(LatData, orient='index')
df_FYSA = df_FYSA.transpose()
df_FYSA.to_excel(writer, sheet_name = 'LatForce VS SlipAng ADJ', index = False)


The next section is just making and showing the plot of the data and saving it to excel.

In [ ]:

plt.xlabel('Slip angle (Deg)')
plt.ylabel('Lateral force (lbs)')
plt.grid()
plt.legend()
plt.savefig(output + '/LateralForceSA.png')
Worksheet1 = writer.sheets['LatForce VS SlipAng ADJ']
Worksheet1.insert_image('A1', output + '/LateralForceSA.png')
plt.show()
